# Puviyan Soil Detection Training - Working Version

Enhanced model with GPU acceleration and TFLite export.

**Instructions:**
1. Runtime → Change runtime type → GPU
2. Run all cells in order

In [ ]:
# Install and import packages
!pip install -q tensorflow matplotlib numpy

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
from google.colab import files

print('TensorFlow version:', tf.__version__)
print('GPU available:', len(tf.config.list_physical_devices('GPU')) > 0)

In [ ]:
# Create synthetic soil data
def generate_soil_data(num_samples):
    # Generate random soil-like images
    X = np.random.randint(50, 200, (num_samples, 224, 224, 3), dtype=np.uint8)
    # Add texture
    for i in range(num_samples):
        noise = np.random.normal(0, 15, X[i].shape)
        X[i] = np.clip(X[i] + noise, 0, 255).astype(np.uint8)
    
    # Generate labels for 8 soil types
    y = np.random.randint(0, 8, num_samples)
    return X, y

# Generate training and validation data
print('Generating training data...')
X_train, y_train = generate_soil_data(4000)
X_val, y_val = generate_soil_data(1000)

print(f'Training data shape: {X_train.shape}')
print(f'Validation data shape: {X_val.shape}')
print(f'Number of classes: {len(np.unique(y_train))}')

In [ ]:
# Create enhanced soil classification model
def create_soil_model():
    model = keras.Sequential([
        # Input preprocessing
        layers.Rescaling(1./255, input_shape=(224, 224, 3)),
        
        # Data augmentation
        layers.RandomFlip('horizontal'),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.1),
        
        # Feature extraction
        layers.Conv2D(32, 3, activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(2),
        
        layers.SeparableConv2D(64, 3, activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(2),
        
        layers.SeparableConv2D(128, 3, activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(2),
        
        layers.SeparableConv2D(256, 3, activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.GlobalAveragePooling2D(),
        
        # Classification head
        layers.Dense(512, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(256, activation='relu'),
        layers.Dropout(0.3),
        layers.Dense(8, activation='softmax')
    ])
    
    return model

# Create and compile model
model = create_soil_model()
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

print('Model created successfully!')
model.summary()

In [ ]:
# Train the model
print('Starting training...')

# Define callbacks
callbacks = [
    keras.callbacks.EarlyStopping(
        monitor='val_accuracy',
        patience=5,
        restore_best_weights=True
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=3
    )
]

# Train model
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=30,
    batch_size=32,
    callbacks=callbacks,
    verbose=1
)

print('Training completed!')

In [ ]:
# Plot training history
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training')
plt.plot(history.history['val_accuracy'], label='Validation')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training')
plt.plot(history.history['val_loss'], label='Validation')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

# Convert to TFLite
print('Converting to TFLite...')
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

# Save model
model_filename = 'puviyan_soil_model.tflite'
with open(model_filename, 'wb') as f:
    f.write(tflite_model)

print(f'Model saved as {model_filename}')
print(f'Model size: {len(tflite_model) / 1024:.1f} KB')

# Download the model
files.download(model_filename)
print('Model download started!')